In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Final Data.csv')
df

,backers_count,country,created_at,deadline,fx_rate,goal,launched_at,spotlight,staff_pick,state_changed_at,class,Score,Comments,Updates
0,45,US,1577743896,1581179226,1.000000,4500.0,1578328026,True,True,1581179226,1,10.47,0.0,9.0
1,21,US,1573666016,1578252537,1.000000,300.0,1575660537,True,False,1578252537,1,14.33,0.0,3.0
2,146,SE,1574880653,1577905200,0.116116,50000.0,1575459486,True,False,1577905200,1,11.18,0.0,0.0
3,31,US,1574060612,1576664821,1.000000,500.0,1574072821,True,False,1576664821,1,15.96,2.0,4.0
4,9,GB,1571141508,1576414740,1.380491,250.0,1573500464,True,False,1576414740,1,12.80,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64713,51,US,1418766416,1425868872,1.000000,5000.0,1423280472,True,False,1425868872,1,13.92,9.0,7.0
64714,11,US,1493913962,1497110947,1.000000,60000.0,1494518947,False,False,1497110948,0,16.24,1.0,0.0
64715,18,US,1427324752,1427677200,1.000000,850.0,1427418491,True,False,1427677215,1,13.46,1.0,4.0
64716,1,GB,1398336316,1401044620,1.406145,10000.0,1398452620,False,False,1401044620,0,13.33,0.0,1.0


In [3]:
X = df.drop(['class','country','spotlight','staff_pick'], axis=1)
Y = df['class']

In [4]:
Y

0        1
1        1
2        1
3        1
4        1
        ..
64713    1
64714    0
64715    1
64716    0
64717    1
Name: class, Length: 64718, dtype: int64

## IG

In [5]:
# from sklearn.feature_selection import mutual_info_classif
# gain = mutual_info_classif(X,Y)
# gain 

## Relief

In [6]:
# import sklearn_relief as sr
# r = sr.RReliefF()
# result = r.fit_transform(X,Y)
# result

## Fisher

In [7]:
# from skfeature.function.similarity_based import fisher_score
# score = fisher_score.fisher_score(df[['backers_count','Score']],df['class'])
# score

## IED

In [8]:
# import numpy as np
# dist = np.linalg.norm(df['backers_count'] - df['Comments'])
# dist

## Pearson Corr

In [9]:
# from scipy.stats import pearsonr
# corr, _ = pearsonr(df['backers_count'], df['Comments']) 
# corr

## Cosine Similarity

In [10]:
# from scipy import spatial
# result = 1 - spatial.distance.cosine(df['backers_count'], df['Comments'])
# result

In [28]:
def HSF(df, df_train):
    
    from copy import deepcopy
    col_list = list(df_train.columns)
    print(col_list)
    d = df_train.shape[1]
    SF = []
    merit = [0 for i in range(d)]
    merit_archive = [0 for i in range(d)]
    Total_FS = [0 for i in range(d)]

    rel_CF = RHFS(df, d, col_list)

    max_value = max(rel_CF)
    Feature_max_index = rel_CF.index(max_value)

    n = 0

    SF.append(Feature_max_index)

    merit_archive[n] = deepcopy(max_value)

    Total_FS[n] = deepcopy(SF)

    while n < d-1:

        for i in range(d):

            red_FF = SHFS(SF, df_train, i, col_list,d)

            top = (d + d*(sum(rel_CF)/len(rel_CF)))
            # bottom = (d + (d*(d-1)*(sum(red_FF)/len(red_FF))))**(1/2)
            bottom = (d - (sum(red_FF)/len(red_FF)))**(1/2)

            merit[i] = top/bottom

        for val in SF:
            merit[val] = -9999999

        max_value = max(merit)
        Feature_max_index = merit.index(max_value)

        if Feature_max_index not in SF:
            SF.append(Feature_max_index)

        n += 1
        
        Total_FS[n] = deepcopy(SF)

        merit_archive[n] = max_value

    index_val = merit_archive.index(max(merit_archive))

    Feature_subset = Total_FS[index_val]

    subset_list = []

    for val in Feature_subset:
        subset_list.append(col_list[val])
    return subset_list


def RHFS(df, d, col_list):
    rel_CF = [0 for i in range(d)]
    count = 0

    igain = IG(X,df['class'])

    for m in  range(d):
        mu = df[col_list[m]].mean()

        n_1 = df[df['class'] == 0].count()[1]
        mu_1 = df[col_list[m]][df['class'] == 0].mean()
        var_1 = df[col_list[m]][df['class'] == 0].var()

        n_2 = df[df['class'] == 1].count()[1]
        mu_2 = df[col_list[m]][df['class'] == 1].mean()
        var_2 = df[col_list[m]][df['class'] == 1].var()

        inter_class = n_1*(mu_1-mu)**2 + n_2*(mu_2-mu)**2
        intra_class = (n_1-1)*var_1 + (n_2-1)*var_2

        fscore = inter_class / intra_class

        rel_CF[count] = (1/2)*(fscore + igain[m])
        count += 1
    
    return rel_CF

        
        

def IG(col_1, col_2):
    from sklearn.feature_selection import mutual_info_classif
    gain = mutual_info_classif(col_1,col_2)
    return gain   

def relief(col_1,col_2):
    pass



def SHFS(SF, df_train, i, col_list,d):
    from itertools import combinations
    from copy import deepcopy
    red_FF = []
    temp_SF = deepcopy(SF)
    if i not in SF:
        temp_SF.append(i)
        P_pairs = list(combinations(temp_SF,2))
        for pair in P_pairs:

            ied_value = (IED(df_train[col_list[pair[0]]],df_train[col_list[pair[1]]]))**2
            pc_value = (PC(df_train[col_list[pair[0]]],df_train[col_list[pair[1]]]))**2
            cs_value = (CS(df_train[col_list[pair[0]]],df_train[col_list[pair[1]]]))**2

            red_FF.append((1/3)*(ied_value + pc_value + cs_value))
            # red_FF.append((1/2)*(pc_value + cs_value))
    else:
        red_FF = [0]
    return red_FF

def IED(col_1, col_2):
    import numpy as np
    dist = np.linalg.norm(col_1 - col_2)
    return 1/dist

def PC(col_1, col_2):
    from scipy.stats import pearsonr
    corr, _ = pearsonr(col_1 , col_2)
    return corr

def CS(col_1, col_2):
    from scipy import spatial
    similarity = 1 - spatial.distance.cosine(col_1, col_2)
    return similarity


In [29]:
feat_list = HSF(df,X)
feat_list

['backers_count', 'created_at', 'deadline', 'fx_rate', 'goal', 'launched_at', 'state_changed_at', 'Score', 'Comments', 'Updates']


['Updates', 'state_changed_at', 'deadline', 'launched_at', 'created_at']